In [1]:
### Make the charts

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import json
from pandas.io.json import json_normalize

In [2]:
# Replace this with scrapers for top IV, top vol, and earnings.

symbol_list = ['SPY', 'FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG', 'IQ', 'TSLA', 'TLT', 'GLD', 'SLV']
#symbol_list = ['TLT', 'FXI']

url = 'https://query2.finance.yahoo.com/v7/finance/options/'

timestamp = datetime.datetime.now().strftime('%Y%m%d')

In [8]:
# Gets the third friday exp

def is_third_friday(s):
    d = datetime.datetime.strptime(s, '%Y-%m-%d')
    return d.weekday() == 3 and 15 <= d.day <= 21

# Gets expiration dates (in unix), converts to human time, gets the first weekly and two monthly dates, gets unix/human time for those.

def date_lister(symbol, exp):
    
    # Find better way to put unix and human dates together, reference the three dates you want.
    
    global dates_list
    
    # Make df for unix and human exp dates
    
    exp_df = pd.DataFrame()  
    exp_df['unix'] = exp[0].values
    
    # Get human values for exp_df
    
    ht_list = []
    
    for i in exp[0]:
        
        ht = datetime.datetime.fromtimestamp(int(i)).strftime('%Y-%m-%d')
        ht_list.append(ht)
        
    exp_df['human'] = ht_list
        
    # Get your nonweekly dates
    
    non_weeklies = []
    
    for date in ht_list:
        if is_third_friday(date) == True:
            non_weeklies.append(date)
            
        else:
            pass
        
    # Make list with first weekly and two nonweeklies, with unix and human time format
    
    dates_list = []
    
    dates_list.append(ht_list[0])
    
    for i in non_weeklies[:2]:
        dates_list.append(i)
    
    dates_list = pd.DataFrame(data = dates_list, columns = ['human'])
    
    dates_list['unix'] = [(exp_df.loc[exp_df['human'] == i, 'unix'].values[0]) for i in dates_list['human']]

def scraper(symbol):
    
    global oc_df
    
    optionschain = []
    
    for date in dates_list['unix']:
    
        newurl2 = newurl + '?date=' + str(date)

        # Temporary df's for our shit, written to different df later for output
        
        temp_df = pd.read_json(newurl2)

        chain = json_normalize(data=temp_df['optionChain']['result'], record_path='options')

        for j in chain.calls[0]:

            j.update({'type': 'call'})
            optionschain.append(j)

        for k in chain.puts[0]:

            k.update({'type': 'put'})
            optionschain.append(k)

        oc_df = pd.DataFrame(optionschain)
        
        #print(oc_df)
        
        ### Add mid, add DTE ###
        
    oc_df['mid'] = (oc_df['bid'] + oc_df['ask']) / 2
        
def graph(symbol):
    
    # Figure out how to graph bid/ask together here
    
    for j in ['bid', 'impliedVolatility']:
        
        plt.figure(figsize = (20,10))
        
        colors = np.linspace(1, .25, len(dates_list))
        
        #print(str(symbol) + ' ' + str(j))
        
        for i, k in zip(dates_list['unix'], colors):
            
            iv = oc_df[oc_df['expiration'] == i]
            
            iv_put = iv[iv['type'] == 'put']
            iv_call = iv[iv['type'] == 'call']

            otm_put = iv_put[iv_put['inTheMoney'] == False]
            otm_call = iv_call[iv_call['inTheMoney'] == False]

            plt.plot(otm_call[['strike', j]].set_index('strike'), label = str(i) + ' call', color=plt.cm.Oranges(k), lw=3)
            plt.plot(otm_put[['strike', j]].set_index('strike'), label = str(i) + ' put', color=plt.cm.Blues(k), lw=3)
            
        # Dynamically size plots here
            
        #plt.ylim(0, 1.75)
        #plt.xlim(200, 350)
        
        plt.legend()
        #plt.show()
        
        filename = timestamp + '_' + symbol + '_' + j + '.png'
        
        filenamelist.append(filename)
        
        plt.savefig(filename)
        plt.gcf().clear()
        plt.close()

In [12]:
# Symbol loop

def chart_maker(symbol):
    
    global newurl
    
    newurl = url + symbol
    
    df = pd.read_json(newurl)
    
    exp = json_normalize(data = df['optionChain']['result'], record_path = 'expirationDates')
    
    date_lister(symbol, exp)
    
    scraper(symbol)
    
    graph(symbol)

In [13]:
def actual_chart_maker_loop(symbol_list):
    
    global filenamelist
    
    filenamelist = []

    for symbol in symbol_list:
    
        chart_maker(symbol)
        

In [14]:
actual_chart_maker_loop(symbol_list)

In [15]:
oc_df

,ask,bid,change,contractSize,contractSymbol,currency,expiration,impliedVolatility,inTheMoney,lastPrice,lastTradeDate,openInterest,percentChange,strike,type,volume,mid
0,4.60,4.55,0.00,REGULAR,SLV180803C00010000,USD,1533254400,1.125004,True,4.56,1532728724,2,0.000000,10.0,call,2,4.575
1,2.60,2.56,0.00,REGULAR,SLV180803C00012000,USD,1533254400,0.687503,True,2.55,1532728725,0,0.000000,12.0,call,10,2.580
2,2.13,2.09,0.00,REGULAR,SLV180803C00012500,USD,1533254400,0.734378,True,2.25,1532144889,20,0.000000,12.5,call,20,2.110
3,1.60,1.56,0.00,REGULAR,SLV180803C00013000,USD,1533254400,0.546880,True,1.60,1532453275,141,0.000000,13.0,call,1,1.580
4,1.11,1.07,0.00,REGULAR,SLV180803C00013500,USD,1533254400,0.429693,True,1.17,1532441618,563,0.000000,13.5,call,2,1.090
5,0.60,0.59,0.08,REGULAR,SLV180803C00014000,USD,1533254400,0.250007,True,0.61,1532708326,425,15.094349,14.0,call,43,0.595
6,0.17,0.16,0.02,REGULAR,SLV180803C00014500,USD,1533254400,0.164071,True,0.16,1532721495,1833,14.285712,14.5,call,1617,0.165
7,0.03,0.02,-0.01,REGULAR,SLV180803C00015000,USD,1533254400,0.199227,False,0.02,1532721245,7800,-33.333336,15.0,call,1924,0.025
8,0.01,0.00,0.00,REGULAR,SLV180803C00015500,USD,1533254400,0.265632,False,0.01,1532622257,2368,0.000000,15.5,call,88,0.005
9,0.01,0.00,0.00,REGULAR,SLV180803C00016000,USD,1533254400,0.367194,False,0.01,1532632424,986,0.000000,16.0,call,15,0.005


In [22]:
### Send the charts

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

fromaddr = 'pythonoptionsalerts@gmail.com'
toaddr = 'samchakerian@gmail'
msg = MIMEMultipart()
msg['From'] = fromaddr
msg['To'] = toaddr
msg['Subject'] = datetime.datetime.now().strftime('%Y-%m-%d') + ' Options Charts' # Put day and options alert type here

body = 'Print sick as fuck stats here.'

msg.attach(MIMEText(body, 'plain'))

for j in filenamelist:

    attachment = open(j, 'rb')

    part = MIMEBase('application', 'octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)

    part.add_header('Content-Disposition', 'attachment; filename= %s' % j)

    msg.attach(part)

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(fromaddr, '4r3e2w1q5t')

text = msg.as_string()
server.sendmail('pythonoptionsalerts@gmail.com', 'samchakerian@gmail.com', text)
server.quit()

(221, b'2.0.0 closing connection i186-v6sm7076997pfc.27 - gsmtp')